### Beamformers
* A unified view on beamformers for M/EEG source reconstruction; Britta U. Westner, Sarang S. Dalal, Alexandre Gramfort, Vladimir Litvak, John C. Mosher, Robert Oostenveld, Jan-Mathijs Schoffelen

### Source space/Sensor space 
* Ultra-Rapid serial visual presentation reveals dynamics of feedforward and feedback processes in the ventral visual pathway; Yalda Mohsenzadeh, Sheng Qin, Radoslaw M Cichy, Dimitrios Pantazis
* MEG-based decoding of the spatiotemporal dynamics of visual category perception; M.E. van de Nieuwenhuijzen, A.R. Backus, A. Bahramisharif, C.F. Doeller, O. Jensen, M.A.J. van Gerven

### Q/A
* https://mne.discourse.group/t/best-practice-to-get-niml-gifti-or-nifti-from-stc/5767/6

In [ ]:
%matplotlib qt

import platform
import os
from os import listdir
import copy
import ast

import numpy as np
import scipy
import scipy.io
import nibabel
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr 
from matplotlib.backends.backend_pdf import PdfPages
from scipy import sparse
from nibabel.funcs import concat_images
from mayavi import mlab

import mne
from mne.io import read_raw_fif, read_info
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.transforms import apply_trans
from mne.surface import read_surface, _compute_nearest
from mne.coreg import Coregistration

matplotlib.use('Qt5Agg')

In [ ]:
# Set intial parameters

tmin = -0.1
tmax = 0.8
baseline = (-0.1,0)

downsample_freq = 200
powerline_freq = (50, 100, 150, 200)
low_pass_freq = 0.05
high_pass_freq = 330

# Savitzky-Golay filter
window_length = 25
polyorder = 4
# Butterworth filter
l_freq = 0.5
h_freq = 40
order = 2

# Number of runs
n_runs = 12

# Subject
# Structural image not available: Subj06, Subj07, Subj10, Subj41
# Automatic: Subj02, Subj03, Subj09, Subj10, Subj11, Subj12, Subj13, Subj14, Subj15, Subj16,
#            Subj17, Subj18, Subj19, Subj20, Subj22, Subj23, Subj24, Subj26, Subj27, Subj28
#            Subj29, Subj30, Subj31, Subj32, Subj34, Subj36, Subj37, Subj38, Subj39, Subj40
#            Subj41, Subj42
# Manual: Subj04 (EOG, ECG), Subj05 (EOG), Subj06 (EOG), Subj07 (EOG, ECG)
# Empty room not available: Subj02, Subj03, Subj04
# Excessive Head Motion: Subj11, Subj17, Subj22, Subj24, Subj29, Subj32
# With oct3, ico3 for source localization: Subj28
subj = "Subj02"

# Events
event_sample_list = ['N1S1TFA1', 'N1S1TFA2', 'N1S2TFA1', 'N1S2TFA2',
                      'N1S3TFA1', 'N1S3TFA2', 'N1S4TFA1', 'N1S4TFA2',
                      'N2S1TFA1', 'N2S1TFA2', 'N2S2TFA1', 'N2S2TFA2',
                      'N2S3TFA1', 'N2S3TFA2', 'N2S4TFA1', 'N2S4TFA2',
                      'N3S1TFA1', 'N3S1TFA2', 'N3S2TFA1', 'N3S2TFA2',
                      'N3S3TFA1', 'N3S3TFA2', 'N3S4TFA1', 'N3S4TFA2',
                      'N4S1TFA1', 'N4S1TFA2', 'N4S2TFA1', 'N4S2TFA2',
                      'N4S3TFA1', 'N4S3TFA2', 'N4S4TFA1', 'N4S4TFA2']

event_sample_dict = {'N1S1TFA1':1, 'N1S1TFA2':2, 'N1S2TFA1':3, 'N1S2TFA2':4,
                     'N1S3TFA1':5, 'N1S3TFA2':6, 'N1S4TFA1':7, 'N1S4TFA2':8,
                     'N2S1TFA1':9, 'N2S1TFA2':10, 'N2S2TFA1':11, 'N2S2TFA2':12,
                     'N2S3TFA1':13, 'N2S3TFA2':14, 'N2S4TFA1':15, 'N2S4TFA2':16,
                     'N3S1TFA1':17, 'N3S1TFA2':18, 'N3S2TFA1':19, 'N3S2TFA2':20,
                     'N3S3TFA1':21, 'N3S3TFA2':22, 'N3S4TFA1':23, 'N3S4TFA2':24,
                     'N4S1TFA1':25, 'N4S1TFA2':26, 'N4S2TFA1':27, 'N4S2TFA2':28,
                     'N4S3TFA1':29, 'N4S3TFA2':30, 'N4S4TFA1':31, 'N4S4TFA2':32}

event_sample_dict_N = {'N1/N1S1TFA1':1, 'N1/N1S1TFA2':2, 'N1/N1S2TFA1':3, 'N1/N1S2TFA2':4,
                      'N1/N1S3TFA1':5, 'N1/N1S3TFA2':6, 'N1/N1S4TFA1':7, 'N1/N1S4TFA2':8,
                      'N2/N2S1TFA1':9, 'N2/N2S1TFA2':10, 'N2/N2S2TFA1':11, 'N2/N2S2TFA2':12,
                      'N2/N2S3TFA1':13, 'N2/N2S3TFA2':14, 'N2/N2S4TFA1':15, 'N2/N2S4TFA2':16,
                      'N3/N3S1TFA1':17, 'N3/N3S1TFA2':18, 'N3/N3S2TFA1':19, 'N3/N3S2TFA2':20,
                      'N3/N3S3TFA1':21, 'N3/N3S3TFA2':22, 'N3/N3S4TFA1':23, 'N3/N3S4TFA2':24,
                      'N4/N4S1TFA1':25, 'N4/N4S1TFA2':26, 'N4/N4S2TFA1':27, 'N4/N4S2TFA2':28,
                      'N4/N4S3TFA1':29, 'N4/N4S3TFA2':30, 'N4/N4S4TFA1':31, 'N4/N4S4TFA2':32}

event_sample_dict_TFA = {'TFA1/N1S1TFA1':1, 'TFA2/N1S1TFA2':2, 'TFA1/N1S2TFA1':3, 'TFA2/N1S2TFA2':4,
                        'TFA1/N1S3TFA1':5, 'TFA2/N1S3TFA2':6, 'TFA1/N1S4TFA1':7, 'TFA2/N1S4TFA2':8,
                        'TFA1/N2S1TFA1':9, 'TFA2/N2S1TFA2':10, 'TFA1/N2S2TFA1':11, 'TFA2/N2S2TFA2':12,
                        'TFA1/N2S3TFA1':13, 'TFA2/N2S3TFA2':14, 'TFA1/N2S4TFA1':15, 'TFA2/N2S4TFA2':16,
                        'TFA1/N3S1TFA1':17, 'TFA2/N3S1TFA2':18, 'TFA1/N3S2TFA1':19, 'TFA2/N3S2TFA2':20,
                        'TFA1/N3S3TFA1':21, 'TFA2/N3S3TFA2':22, 'TFA1/N3S4TFA1':23, 'TFA2/N3S4TFA2':24,
                        'TFA1/N4S1TFA1':25, 'TFA2/N4S1TFA2':26, 'TFA1/N4S2TFA1':27, 'TFA2/N4S2TFA2':28,
                        'TFA1/N4S3TFA1':29, 'TFA2/N4S3TFA2':30, 'TFA1/N4S4TFA1':31, 'TFA2/N4S4TFA2':32}

event_sample_dict_S = {'S1/N1S1TFA1':1, 'S1/N1S1TFA2':2, 'S2/N1S2TFA1':3, 'S2/N1S2TFA2':4,
                      'S3/N1S3TFA1':5, 'S3/N1S3TFA2':6, 'S4/N1S4TFA1':7, 'S4/N1S4TFA2':8,
                      'S1/N2S1TFA1':9, 'S1/N2S1TFA2':10, 'S2/N2S2TFA1':11, 'S2/N2S2TFA2':12,
                      'S3/N2S3TFA1':13, 'S3/N2S3TFA2':14, 'S4/N2S4TFA1':15, 'S4/N2S4TFA2':16,
                      'S1/N3S1TFA1':17, 'S1/N3S1TFA2':18, 'S2/N3S2TFA1':19, 'S2/N3S2TFA2':20,
                      'S3/N3S3TFA1':21, 'S3/N3S3TFA2':22, 'S4/N3S4TFA1':23, 'S4/N3S4TFA2':24,
                      'S1/N4S1TFA1':25, 'S1/N4S1TFA2':26, 'S2/N4S2TFA1':27, 'S2/N4S2TFA2':28,
                      'S3/N4S3TFA1':29, 'S3/N4S3TFA2':30, 'S4/N4S4TFA1':31, 'S4/N4S4TFA2':32}

event_dict_N = {'N1':1, 'N2':2, 'N3':3, 'N4':4, 'Smaller Match':5, 'Larger Match':6, 'Left Button':7, 'Right Button':8}

event_dict_S = {'S1':1, 'S2':2, 'S3':3, 'S4':4, 'Smaller Match':5, 'Larger Match':6, 'Left Button':7, 'Right Button':8}

event_dict_TFA = {'TFA1':1, 'TFA2':2, 'Smaller Match':3, 'Larger Match':4, 'Left Button':5, 'Right Button':6}

In [ ]:
def set_path(subj):
    # Check Platform
    if platform.system() == 'Windows':
        project_path = 'F:\\Data Fusion Project'
        subject_folder = 'subject\\'+subj
    elif platform.system() == 'Linux':
        project_path = '/media/sf_Data_Fusion_Project'
        subject_folder = 'subject/'+subj

    # Tail specify the hyperparameter (e.g. ica, autoreject, smoothing, ...)
    tail_fname = "_ica"+"_savgol-"+str(window_length)+"-"+str(polyorder)+"_downsample-"+str(downsample_freq)
    #tail_fname = "_ica"+"_butter-"+str(order)+"-"+str(l_freq)+"-"+str(h_freq)+"_downsample-"+str(downsample_freq)
    #tail_fname = "_ica"+"_downsample-"+str(downsample_freq)
    tail_empty_fname = "_savgol-"+str(window_length)+"-"+str(polyorder)+"_downsample-"+str(downsample_freq)
    #tail_empty_fname = "_downsample-"+str(downsample_freq)
    raw_fname = "raw_tsss_trans.fif"
    epoched_fname = "all"+tail_fname+"-epo.fif"
    log_fname = "log_source"+tail_fname+".txt"
    information_fname = "information.mat"
    t1_fname = "T1.mgz"
    transformation_fname = subj+"-trans.fif"

    data_folder = "data"
    mri_folder = "mri"
    meg_folder = "meg"
    figure_folder = "fig"
    volume_folder = "source"
    information_folder = "information"
    empty_room_folder = "empty"
    freesurfer_folder = "freesurfer"
    fsaverage_folder = "freesurfer\\fsaverage"

    mri_path = os.path.join(project_path,subject_folder,mri_folder)
    freesurfer_path = os.path.join(project_path,subject_folder,mri_folder)
    meg_path = os.path.join(project_path,subject_folder,meg_folder)
    volume_path = os.path.join(project_path,subject_folder,meg_folder,volume_folder)
    log_path = os.path.join(project_path,subject_folder,meg_folder)
    transformation_path = os.path.join(project_path,subject_folder,meg_folder)
    information_path = os.path.join(project_path,data_folder,information_folder)
    empty_room_path = os.path.join(project_path,data_folder,empty_room_folder)
    fsaverage_path = os.path.join(project_path,data_folder,fsaverage_folder)
    figure_path = os.path.join(project_path,subject_folder,meg_folder,figure_folder)
    
    globals().update(locals())

In [ ]:
def save_multi_image(figs,fname):
    pdf = PdfPages(fname)
    for fig in range(len(figs)):
        pdf.savefig(figs[fig])
    pdf.close()

In [ ]:
# Import data and information
#https://mne.tools/stable/auto_tutorials/intro/plot_10_overview.html

#https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries
#https://stackoverflow.com/questions/59635318/load-a-mat-file-including-string-array-to-python-3-6

def import_data():
    path = os.path.join(meg_path,"%i_"+raw_fname)
    raws = [read_raw_fif(path % run, verbose='error')
            for run in range(1,n_runs+1)]  # ignore filename warnings
    
    return raws

#--------------------------------------------------#

def import_info():
    def loadmat(filename):
        '''
        this function should be called instead of direct spio.loadmat
        as it cures the problem of not properly recovering python dictionaries
        from mat files. It calls the function check keys to cure all entries
        which are still mat-objects
        '''
        def _check_keys(d):
            '''
            checks if entries in dictionary are mat-objects. If yes
            todict is called to change them to nested dictionaries
            '''
            for key in d:
                if isinstance(d[key], scipy.io.matlab.mio5_params.mat_struct):
                    d[key] = _todict(d[key])
            return d

        def _todict(matobj):
            '''
            A recursive function which constructs from matobjects nested dictionaries
            '''
            d = {}
            for strg in matobj._fieldnames:
                elem = matobj.__dict__[strg]
                if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
                    d[strg] = _todict(elem)
                elif isinstance(elem, np.ndarray):
                    d[strg] = _tolist(elem)
                else:
                    d[strg] = elem
            return d

        def _tolist(ndarray):
            '''
            A recursive function which constructs lists from cellarrays
            (which are loaded as numpy ndarrays), recursing into the elements
            if they contain matobjects.
            '''
            elem_list = []
            for sub_elem in ndarray:
                if isinstance(sub_elem, scipy.io.matlab.mio5_params.mat_struct):
                    elem_list.append(_todict(sub_elem))
                elif isinstance(sub_elem, np.ndarray):
                    elem_list.append(_tolist(sub_elem))
                else:
                    elem_list.append(sub_elem)
            return elem_list
        data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
        return _check_keys(data)
    
    path = os.path.join(information_path,information_fname)
    subject_info = loadmat(path)
    
    return subject_info

In [ ]:
# Concatenate raw objects
# I reimplement this function from "concatenate_raws" in MNE because the function changed raws[0]
#https://github.com/mne-tools/mne-python/blob/maint/0.24/mne/io/base.py#L2490-L2533

def concatenate_raw_obj(raws, on_mismatch='raise'):
    
    from mne.io.meas_info import _ensure_infos_match
    
    # Make a deepcopy of raws to keep raws intact
    #https://robertheaton.com/2014/02/09/pythons-pass-by-object-reference-as-explained-by-philip-k-dick/
    _raws = copy.deepcopy(raws)
    
    for idx, r in enumerate(_raws[1:], start=1):
        _ensure_infos_match(info1=_raws[0].info, info2=r.info, name=f'raws[{idx}]', on_mismatch=on_mismatch)
        
    _raws[0].append(_raws[1:])
    
    return _raws[0]

In [ ]:
# Modifying raw objects
#https://mne.tools/stable/auto_tutorials/raw/10_raw_overview.html

def modify_raw_obj(raws, subject_info):
    for r in range(1,n_runs+1):
        drop_channels = subject_info['meg'][subj]['channels']['drop']['run'+str(r)]
        rename_channels = subject_info['meg'][subj]['channels']['rename']['run'+str(r)]

        if rename_channels != 'none':
            channel_dict = ast.literal_eval(rename_channels)
            for c in channel_dict.items():
                try:
                    raws[r-1].rename_channels(dict([c]))
                except:
                    raws[r-1].drop_channels(list(dict([c]).values()))
                    raws[r-1].rename_channels(dict([c]))
        if drop_channels != 'none':
            raws[r-1].drop_channels(drop_channels)

        if not 'EOG061' in subject_info['meg'][subj]['channels']['drop']['run'+str(r)]:
            raws[r-1].set_channel_types({'EOG061':'eog'})
        if not 'EOG062' in subject_info['meg'][subj]['channels']['drop']['run'+str(r)]:
            raws[r-1].set_channel_types({'EOG062':'eog'})
        if not 'ECG063' in subject_info['meg'][subj]['channels']['drop']['run'+str(r)]:
            raws[r-1].set_channel_types({'ECG063':'ecg'})

In [ ]:
# Generating annotations programmatically - Muscle activity
#https://mne.tools/dev/auto_examples/preprocessing/muscle_detection.html

def gen_annot_muscle(raw):
    _raw = raw.copy()
    _raw.load_data()

    # The threshold is data dependent, check the optimal threshold by plotting muscle_scores.
    muscle_threshold = 5  # z-score
    # Choose one channel type, if there are axial gradiometers and magnetometers
    # Select magnetometers as they are more sensitive to muscle activity
    muscle_annot, muscle_scores = mne.preprocessing.annotate_muscle_zscore(_raw, ch_type="mag", threshold=muscle_threshold, min_length_good=0.2, filter_freq=[110, 140])

    _raw.set_annotations(muscle_annot)
    
    return _raw

In [ ]:
# Filtering data - Power line noise
#https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html

def filter_power_noise(raw):
    """
    def add_arrows(axes):
        # add some arrows at 50 Hz and its harmonics
        for ax in axes:
            freqs = ax.lines[-1].get_xdata()
            psds = ax.lines[-1].get_ydata()
            for freq in freqs:
                idx = np.searchsorted(freqs, freq)
                # get ymax of a small region around the freq. of interest
                y = psds[(idx - 4):(idx + 5)].max()
                ax.arrow(x=freqs[idx], y=y + 18, dx=0, dy=-12, color='red',
                         width=0.1, head_width=3, length_includes_head=True)
    """

    _raw = raw.copy()
    _raw.load_data()

    meg_picks = mne.pick_types(_raw.info, meg=True)
    freqs = powerline_freq
    _raw.notch_filter(freqs=freqs, picks=meg_picks)
    
    return _raw

In [ ]:
# Filtering data - Band-pass filter
#https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html
#https://jasmainak.github.io/mne-workshop-brown/preprocessing/filtering.html

def filter_band(raw):
    _raw = raw.copy()
    _raw.load_data()

    meg_picks = mne.pick_types(_raw.info, meg=True)
    _raw.filter(low_pass_freq, high_pass_freq, fir_design='firwin', picks=meg_picks)

    return _raw

In [ ]:
# Smooth data
#https://www.mdpi.com/1424-8220/20/3/807
#https://www.biorxiv.org/content/10.1101/731687v1.full
#https://brainder.org/2011/08/20/gaussian-kernels-convert-fwhm-to-sigma/
#https://www.delftstack.com/howto/python/smooth-data-in-python/    

def smooth_savgol_raw(raw):
    #Savitzky-Golay filter
    def savgol(x):
        return scipy.signal.savgol_filter(x,window_length,polyorder)

    _raw = raw.copy()
    _raw.load_data()

    meg_picks = mne.pick_types(_raw.info, meg=True)
    _raw.apply_function(savgol, picks=meg_picks, dtype=None, n_jobs=1, channel_wise=True)

    return _raw

In [ ]:
# Smooth data (Butterworth Filter)
#https://mne.discourse.group/t/butterworth-filter/5760

def smooth_butter_raw(raw):
    _raw = raw.copy()
    _raw.load_data()

    iir_params = dict(order=order, ftype='butter')
    meg_picks = mne.pick_types(_raw.info, meg=True)
    _raw.filter(l_freq=l_freq, h_freq=h_freq, picks=meg_picks, method='iir', iir_params=iir_params, verbose=True)

    return _raw

In [ ]:
# Downsampling
#https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html
#https://jasmainak.github.io/mne-workshop-brown/preprocessing/filtering.html

def downsample_raw(raw):
    #raw.resample(120,npad="auto") # set sampling frequency to 120Hz

    current_sfreq = raw.info['sfreq']
    desired_sfreq = downsample_freq
    decim = np.round(current_sfreq / desired_sfreq).astype(int)
    obtained_sfreq = current_sfreq / decim
    lowpass_freq = obtained_sfreq / 3.

    _raw = raw.copy()
    _raw.load_data()

    _raw.filter(l_freq=None, h_freq=lowpass_freq)

    return _raw

In [ ]:
# Repairing artifacts with ICA
#https://mne.tools/stable/auto_tutorials/preprocessing/40_artifact_correction_ica.html
#https://labeling.ucsd.edu/tutorial/labels

def fit_ica(raw,r):
    figs = []

    # Filtering to remove slow drifts
    # As filtering is a linear operation, the ICA solution found from the filtered signal can be applied to the unfiltered signal
    _raw = raw.copy()
    _raw.load_data().filter(l_freq=1., h_freq=None)

    # Estimate noise covariance matrix from a continuous segment of raw data.
    # It is typically useful to estimate a noise covariance from empty room data or time intervals before starting the stimulation.
    #noise_cov = mne.compute_raw_covariance(raw_erm, tmin=0, tmax=None)

    # Fitting the ICA solution
    ica = ICA(n_components=30, max_iter=1000, random_state=97, method='picard')
    #ica = ICA(n_components=15, max_iter=1000, random_state=97, method='picard', noise_cov=noise_cov)
    ica.fit(_raw)
    
    figs.append(ica.plot_sources(raw, show_scrollbars=False))
    figs.append(ica.plot_components(picks=slice(0,30,1)))

    ecg_idx = mne.pick_types(raw.info, meg=False, eeg=False, stim=False,
                             eog=False, ecg=True, emg=False, ref_meg=False,
                             exclude='bads')
    
    eog_idx = mne.pick_types(raw.info, meg=False, eeg=False, stim=False,
                             eog=True, ecg=False, emg=False, ref_meg=False,
                             exclude='bads')
    
    if list(ecg_idx)!=[]:
        ecg_evoked = create_ecg_epochs(raw).average()
        ecg_evoked.apply_baseline(baseline=(None, -0.2))
        ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='correlation', threshold='auto')
        ecgAvailable = True
    else:
        ecgAvailable = False
    
    if eog_idx!=[]:
        eog_evoked = create_eog_epochs(raw).average()
        eog_evoked.apply_baseline(baseline=(None, -0.2))
        eog_indices, eog_scores = ica.find_bads_eog(raw)
        eogAvailable = True
    else:
        eogAvailable = False

    # Save figures temporarily for inspection
    fname=str(r+1)+'_temp_figs.pdf'
    save_multi_image(figs,os.path.join(figure_path,fname))
    matplotlib.pyplot.close('all')
        
    # If EOG channels are not available, user should select the components manually
    if not eogAvailable:
        eog_indices = []
        print("EOG was not available, consequently, please select the EOG components manually!")
        n = int(input("Please enter the number of EOG component(s) you want to exclude (RUN "+str(r+1) +"): "))
        for i in range(n):
            eog_indices.append(int(input("Please enter the index of the  EOG component(s) you want to exclude one by one: ")))
            
    # If ECG channel is not available, user should select the components manually
    if not ecgAvailable:
        ecg_indices = []
        print("ECG was not available, consequently, please select the ECG components manually!")
        n = int(input("Please enter the number of ECG component(s) you want to exclude (RUN "+str(r+1) +"): "))
        for i in range(n):
            ecg_indices.append(int(input("Please enter the index of the ECG component(s) you want to exclude one by one: ")))
            
    # Delete temporary figures
    os.remove(os.path.join(figure_path,fname))

    if eogAvailable and eog_indices!=[]:
        ica.exclude = []
        ica.exclude = eog_indices

        # Barplot of ICA component "EOG match" scores
        figs.append(ica.plot_scores(eog_scores))

        # Plot diagnostics
        figs = figs + ica.plot_properties(raw, picks=eog_indices)

        # Plot ICs applied to raw data, with EOG matches highlighted
        figs.append(ica.plot_sources(raw, show_scrollbars=False))

        # Plot ICs applied to the averaged EOG epochs, with EOG matches highlighted
        figs.append(ica.plot_sources(eog_evoked))

        ica.exclude = []
        try:
            eog_index = [eog_indices[0]]
        except:
            eog_index = [eog_indices]
    elif eog_indices==[]:
        eog_index = []
    else:
        try:
            eog_index = [eog_indices[0]]
        except:
            eog_index = [eog_indices]

    if ecgAvailable and ecg_indices!=[]:
        ica.exclude = []
        ica.exclude = ecg_indices

        # Barplot of ICA component "ECG match" scores
        figs.append(ica.plot_scores(ecg_scores))

        # Plot diagnostics
        figs = figs + ica.plot_properties(raw, picks=ecg_indices)

        # Plot ICs applied to raw data, with ECG matches highlighted
        figs.append(ica.plot_sources(raw, show_scrollbars=False))

        # Plot ICs applied to the averaged ECG epochs, with ECG matches highlighted
        figs.append(ica.plot_sources(ecg_evoked))

        ica.exclude = []
        try:
            ecg_index = [ecg_indices[0]]
        except:
            ecg_index = [ecg_indices]
    elif ecg_indices==[]:
        ecg_index = []
    else:
        try:
            ecg_index = [ecg_indices[0]]
        except:
            ecg_index = [ecg_indices]

    # Exclude only the first components
    ica.exclude = eog_index+ecg_index
    print("EOG index "+str(eog_index)+" was removed!")
    print("ECG index "+str(ecg_index)+" was removed!")

    if ica.exclude!=[]:
        _raw = raw.copy()
        _raw.load_data()
        ica.apply(_raw)

        #fname=str(r+1)+'_components'+tail_fname+'.pdf'
        #save_multi_image(figs,os.path.join(figure_path,fname))
        matplotlib.pyplot.close('all')
    else:
        _raw = raw.copy()
        
        #fname=str(r+1)+'_components'+tail_fname+'.pdf'
        #save_multi_image(figs,os.path.join(figure_path,fname))
        matplotlib.pyplot.close('all')

    return _raw

In [ ]:
# Make experimental events
#https://mne.tools/stable/auto_tutorials/intro/plot_10_overview.html
#https://mne.tools/dev/auto_tutorials/raw/plot_20_event_arrays.html

def make_event(raw):
    # Sample Stimulus: 1-32
    # Smaller Match Stimulus: 1-32 + 32
    # Larger Match Stimulus: 1-32 + 64
    # Button Pressed: 1,2,4,8 + 128
    # Right Hand: Yellow (2), Green (4)
    # Left Hand: Red (1), Blue (8)
    # 4 comparision in each block (2 larger, 2 smaller)
    # 16 comarision in each fif files (8 larger, 8 smaller)
    
    _raw = raw.copy()
    events = mne.find_events(_raw, stim_channel='STI101', min_duration=.005, shortest_event=1, output='onset')

    # Merged events for number
    merged_events_num = events
    merged_events_num = mne.merge_events(merged_events_num, list(range(1,9)), 1)   #N1
    merged_events_num = mne.merge_events(merged_events_num, list(range(9,17)), 2)   #N2
    merged_events_num = mne.merge_events(merged_events_num, list(range(17,25)), 3)   #N3
    merged_events_num = mne.merge_events(merged_events_num, list(range(25,33)), 4)   #N4
    merged_events_num = mne.merge_events(merged_events_num, list(range(33,65)), 5)   #Smaller Match
    merged_events_num = mne.merge_events(merged_events_num, list(range(65,129)), 6)   #Larger Match
    merged_events_num = mne.merge_events(merged_events_num, [129,136], 7)   #Left Button
    merged_events_num = mne.merge_events(merged_events_num, [130,132], 8)   #Right Button

    # Merged events for size
    merged_events_size = events
    merged_events_size = mne.merge_events(merged_events_size, [1,2,9,10,17,18,25,26], 1)   #S1
    merged_events_size = mne.merge_events(merged_events_size, [3,4,11,12,19,20,27,28], 2)   #S2
    merged_events_size = mne.merge_events(merged_events_size, [5,6,13,14,21,22,29,30], 3)   #S3
    merged_events_size = mne.merge_events(merged_events_size, [7,8,15,16,23,24,31,32], 4)   #S4
    merged_events_size = mne.merge_events(merged_events_size, list(range(33,65)), 5)   #Smaller Match
    merged_events_size = mne.merge_events(merged_events_size, list(range(65,129)), 6)   #Larger Match
    merged_events_size = mne.merge_events(merged_events_size, [129,136], 7)   #Left Button
    merged_events_size = mne.merge_events(merged_events_size, [130,132], 8)   #Right Button

    # Merged events for TFA
    merged_events_tfa = events
    merged_events_tfa = mne.merge_events(merged_events_tfa, [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31], 1)   #TFA1
    merged_events_tfa = mne.merge_events(merged_events_tfa, [2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32], 2)   #TFA2
    merged_events_tfa = mne.merge_events(merged_events_tfa, list(range(33,65)), 3)   #Smaller Match
    merged_events_tfa = mne.merge_events(merged_events_tfa, list(range(65,129)), 4)   #Larger Match
    merged_events_tfa = mne.merge_events(merged_events_tfa, [129,136], 5)   #Left Button
    merged_events_tfa = mne.merge_events(merged_events_tfa, [130,132], 6)   #Right Button

    return events

In [ ]:
# Epoch the data
#https://mne.tools/stable/auto_tutorials/intro/plot_10_overview.html
#https://mne.tools/stable/auto_tutorials/preprocessing/20_rejecting_bad_data.html
#https://www.fieldtriptoolbox.org/tutorial/visual_artifact_rejection/
#https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html

def epoch_data(events,raw):
    
    current_sfreq = raw.info['sfreq']
    desired_sfreq = downsample_freq
    decim = np.round(current_sfreq / desired_sfreq).astype(int)
    
    """
    # Reject criteria
    reject_criteria = dict(mag=4000e-15,     # 4000 fT
                           grad=4000e-13)    # 4000 fT/cm

    # Flat criteria
    flat_criteria = dict(mag=1e-15,          # 1 fT
                         grad=1e-13)         # 1 fT/cm

    # Epochs with rejecting criteria and annotation
    epochs_sample_N = mne.Epochs(raw, events, event_id=event_sample_dict_N,
                                 baseline=baseline, tmin=tmin, tmax=tmax, decim=decim, 
                                 reject=reject_criteria, flat=flat_criteria,
                                 reject_by_annotation=True,
                                 preload=True)

    # Epochs with rejecting annotation
    epochs_sample_N = mne.Epochs(raw, events, event_id=event_sample_dict_N,
                                 baseline=baseline, tmin=tmin, tmax=tmax, decim=decim, 
                                 reject_by_annotation=True,
                                 preload=True)
    """

    # Epochs without rejecting criteria and annotation
    epochs_sample_all = mne.Epochs(raw, events, event_id=event_sample_dict,
                                 baseline=baseline, tmin=tmin, tmax=tmax, decim=decim)
    
    epochs = {"all": epochs_sample_all}

    return epochs

In [ ]:
# FreeSurfer MRI reconstruction
#https://mne.tools/stable/auto_tutorials/source-modeling/plot_background_freesurfer.html

def reconstruct_mri():
    Brain = mne.viz.get_brain_class()
    brain = Brain(freesurfer_folder, hemi='lh', surf='pial', subjects_dir=mri_path, size=(800, 600))
    brain.add_annotation('aparc.a2009s', borders=False)

In [ ]:
# Visualize MRI coordinate frames
#https://mne.tools/stable/auto_tutorials/source-modeling/plot_background_freesurfer_mne.html

def visualize_mri():
    t1 = nibabel.load(os.path.join(mri_path, freesurfer_folder, 'mri', t1_fname))
    t1.orthoview()

In [ ]:
# Make BEM
#https://mne.tools/stable/auto_tutorials/source-modeling/plot_forward.html

# Before running the following command, run the following commands
# Run the following commands in the same command window that anaconda has been launched
#export FREESURFER_HOME=/home/ali/FreeSurfer
#source $FREESURFER_HOME/SetUpFreeSurfer.sh

def make_bem(path_to_freesurfer):
    mne.bem.make_watershed_bem(freesurfer_folder, subjects_dir=path_to_freesurfer)

In [ ]:
# Visualize BEM
#https://mne.tools/stable/auto_tutorials/source-modeling/plot_forward.html

def visualize_bem():
    mne.viz.plot_bem(subject=freesurfer_folder, subjects_dir=mri_path, brain_surfaces='white', orientation='coronal')

In [ ]:
# Make Coregistration (GUI)
#https://pybrain-workshop.github.io/
#https://mne.tools/stable/auto_tutorials/source-modeling/plot_source_alignment.html
#https://www.slideshare.net/mne-python/mnepython-coregistration
#https://www.youtube.com/watch?v=ALV5qqMHLlQ&t=6s
#https://mne-cpp.github.io/pages/documentation/analyze_coregistration.html

def make_coregistration_gui(epoch):
    trans = os.path.join(transformation_path,transformation_fname)
    mne.gui.coregistration(freesurfer_folder, subjects_dir=mri_path, inst=epoch)

In [ ]:
# Make Coregistration (Automatic)
#https://mne.tools/stable/auto_tutorials/forward/25_automated_coreg.html

def make_coregistration_automatic(epoch):
    
    plot_kwargs = dict(subject=freesurfer_folder, subjects_dir=mri_path,
                   surfaces="head", dig=True, eeg=[],
                   meg='sensors', show_axes=True,
                   coord_frame='meg')
    view_kwargs = dict(elevation=90, distance=0.6,
                       focalpoint=(0., 0., 0.))

    # Set up the coregistration model
    info = read_info(epoch)
    fiducials = "auto"  # get fiducials from subject
    coreg = Coregistration(info, subject=freesurfer_folder, subjects_dir=mri_path, fiducials=fiducials)
    #fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

    # Initial fit with fiducials
    coreg.fit_fiducials(verbose=True)
    #fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

    # Refining with ICP
    coreg.fit_icp(n_iterations=20, nasion_weight=10, verbose=True)
    #fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)

    # Omitting bad points
    coreg.omit_head_shape_points(distance=5. / 1000)  # distance is in meters

    # Final coregistration fit
    coreg.fit_icp(n_iterations=20, nasion_weight=10., verbose=True)
    
    # https://mne.discourse.group/t/how-to-save-plot-sensors-connectivity/4958/4
    fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)
    mne.viz.set_3d_view(fig, azimuth=90, **view_kwargs)
    screenshot = fig.plotter.screenshot()
    fig_temp, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(screenshot, origin='upper')
    ax.set_axis_off()  # Disable axis labels and ticks
    fig_temp.tight_layout()
    fig_temp.savefig(os.path.join(figure_path,'coregistration_90.png'), dpi=150)
    matplotlib.pyplot.close('all')
    
    fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)
    mne.viz.set_3d_view(fig, azimuth=45, **view_kwargs)
    screenshot = fig.plotter.screenshot()
    fig_temp, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(screenshot, origin='upper')
    ax.set_axis_off()  # Disable axis labels and ticks
    fig_temp.tight_layout()
    fig_temp.savefig(os.path.join(figure_path,'coregistration_45.png'), dpi=150)
    matplotlib.pyplot.close('all')
    
    fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)
    mne.viz.set_3d_view(fig, azimuth=0, **view_kwargs)
    screenshot = fig.plotter.screenshot()
    fig_temp, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(screenshot, origin='upper')
    ax.set_axis_off()  # Disable axis labels and ticks
    fig_temp.tight_layout()
    fig_temp.savefig(os.path.join(figure_path,'coregistration_0.png'), dpi=150)
    matplotlib.pyplot.close('all')
    
    dists = coreg.compute_dig_mri_distances() * 1e3  # in mm
    print(
        f"Distance between HSP and MRI (mean/min/max):\n{np.mean(dists):.2f} mm "
        f"/ {np.min(dists):.2f} mm / {np.max(dists):.2f} mm"
    )

    # Save the file
    mne.write_trans(meg_path+'\\'+transformation_fname, coreg.trans)

In [ ]:
# Visualize Coregistration
#https://mne.tools/stable/auto_tutorials/source-modeling/plot_source_alignment.html
#https://mne.tools/stable/auto_tutorials/source-modeling/plot_forward.html

def visualize_coregistration(epoch):
    # The transformation file obtained by coregistration
    trans = os.path.join(transformation_path,transformation_fname)
    info = mne.io.read_info(epoch)

    mne.viz.plot_alignment(info, trans, freesurfer_folder, subjects_dir=mri_path, dig=True, meg=['helmet', 'sensors'], surfaces='outer_skull')

In [ ]:
#Computing a Covariance Matrix from Baseline
#https://mne.tools/stable/auto_tutorials/forward/90_compute_covariance.html

def make_cov_baseline(figure_path,event_fname,epoch):
    figs = []
    
    noise_cov = mne.compute_covariance(epoch, tmax=0)
    
    fig = noise_cov.plot(epoch.info, proj=True);
    figs.append(fig[0])
    figs.append(fig[1])
    save_multi_image(figs,os.path.join(figure_path+'\\'+f"cov_matrix_{event_fname}{tail_fname}.pdf"))
    matplotlib.pyplot.close('all')
    
    return noise_cov

In [ ]:
#Computing a Covariance Matrix from Empty Room
#https://mne.tools/stable/auto_tutorials/forward/90_compute_covariance.html

def make_cov_empty(raw_empty_room,raw_empty_room_fname):
    figs = []
    
    noise_cov = mne.compute_raw_covariance(raw_empty_room, tmin=0, tmax=None)
    mne.write_cov(os.path.join(empty_room_path, raw_empty_room_fname.replace('.fif',tail_empty_fname+'_cov.fif.gz')),noise_cov)
    
    fig = noise_cov.plot(raw_empty_room.info, proj=True);
    figs.append(fig[0])
    figs.append(fig[1])
    save_multi_image(figs,os.path.join(empty_room_path,raw_empty_room_fname.replace('.fif',tail_empty_fname+'.pdf')))
    matplotlib.pyplot.close('all')

In [ ]:
#Reading a Covariance Matrix from Empty Room
#https://mne.tools/stable/auto_tutorials/forward/90_compute_covariance.html

def read_cov_empty(subject_info):    
    raw_empty_room_fname = subject_info['meg'][subj]['empty']+tail_empty_fname+"_cov.fif.gz"
    noise_cov = mne.read_cov(os.path.join(empty_room_path, raw_empty_room_fname))
    
    return noise_cov

In [ ]:
#Make Source Localization
#https://mne.tools/stable/auto_tutorials/inverse/30_mne_dspm_loreta.html

def make_source_localization(epoch,evoked,noise_cov):
    trans = os.path.join(transformation_path,transformation_fname)
    src = mne.setup_source_space(subject='freesurfer', spacing='oct4', add_dist='patch', surface='white', subjects_dir=mri_path)
    model = mne.make_bem_model(subject='freesurfer', ico=4, conductivity=(0.3,), subjects_dir=mri_path)
    bem = mne.make_bem_solution(model)
    fwd = mne.make_forward_solution(epoch.info, trans=trans, src=src, bem=bem, meg=True, eeg=False, mindist=5.0, n_jobs=1, verbose=True)
    inverse_operator = make_inverse_operator(evoked.info, fwd, noise_cov, loose=1, depth=None, verbose=True)
    del fwd
    
    method = "dSPM"
    snr = 3.
    lambda2 = 1. / snr ** 2
    stc= apply_inverse(evoked, inverse_operator, lambda2, method=method, pick_ori=None, verbose=True)
    #src = inverse_operator['src']
    
    return stc, src

In [ ]:
#Visualize Source Localization
#https://mne.tools/stable/auto_tutorials/inverse/30_mne_dspm_loreta.html
#https://mne.tools/stable/auto_tutorials/inverse/60_visualize_stc.html

def visualize_source_localization(stc,hemi):
    vertno_max, time_max = stc.get_peak(hemi=hemi)
    surfer_kwargs = dict(hemi=hemi, subjects_dir=mri_path, clim=dict(kind='value', lims=[5, 95, 185]), 
                         views='lateral', initial_time=0, time_unit='s', size=(800, 800), smoothing_steps=15)

    brain = stc.plot(**surfer_kwargs)
    brain.add_foci(vertno_max, coords_as_verts=True, hemi=hemi, color='blue', scale_factor=0.6, alpha=0.5)
    brain.add_text(0.1, 0.9, 'dSPM (plus location of maximal activation)', 'title', font_size=14)
    #brain.save_movie(time_dilation=20, tmin=0.05, tmax=0.16, interpolation='linear', framerate=10)

In [ ]:
#Visualize Source Localization (Morph)
#https://mne.tools/stable/auto_tutorials/inverse/30_mne_dspm_loreta.html
#https://mne.tools/stable/auto_tutorials/inverse/60_visualize_stc.html

def visualize_source_localization_morph(stc):
    stc_fs = mne.compute_source_morph(stc, 'freesurfer', fsaverage_path, mri_path, smooth=15, verbose='error').apply(stc)
    brain = stc_fs.plot(subjects_dir=mri_path, initial_time=0,
                    clim=dict(kind='value', lims=[5, 95, 185]),
                    surface='flat', hemi='both', size=(1000, 500),
                    smoothing_steps=15, time_viewer=True,
                    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=10)))

In [ ]:
#Exporting Surface Sources as NIfTI Volume
#https://mne.discourse.group/t/exporting-surface-sources-as-nifti-volume/5518

def save_surface_src_volume(stc, subject, subjects_dir, time_downsample_factor, save_file_name):
    n_vertices = sum(len(v) for v in stc.vertices)
    offset = 0
    surf_to_mri = 0.
    for hi, hemi in enumerate(['lh', 'rh']):
        ribbon = nib.load(Path(subjects_dir / subject / 'mri' / f'{hemi}.ribbon.mgz'))
        xfm = ribbon.header.get_vox2ras_tkr()
        mri_data = np.asanyarray(ribbon.dataobj)
        ijk = np.array(np.where(mri_data)).T
        xyz = apply_trans(xfm, ijk) / 1000.
        row_ind = np.where(mri_data.ravel())[0]
        data = np.ones(row_ind.size)
        rr = read_surface(Path(subjects_dir / subject / 'surf'/ f'{hemi}.white'))[0]
        rr /= 1000.
        rr = rr[stc.vertices[hi]]
        col_ind = _compute_nearest(rr, xyz) + offset
        surf_to_mri = surf_to_mri + sparse.csr_matrix(
            (data, (row_ind, col_ind)), shape=(mri_data.size, n_vertices))
        offset += len(stc.vertices[hi])

    source_data = xr.DataArray(stc.data, dims=["sources", "time"]) \
                    .rolling(time=time_downsample_factor, center=True) \
                    .mean()[:, time_downsample_factor//2:-time_downsample_factor//2:time_downsample_factor]

    data_all_times = []
    for time_data in source_data.transpose("time", "sources").values:
        data = surf_to_mri.dot(time_data)
        data = data.reshape(mri_data.shape).astype("float32")
        data_all_times.append(nib.Nifti1Image(data, ribbon.affine))

    nib.save(concat_images(data_all_times), save_file_name)

In [ ]:
########## Make BEM ##########
subject_list = ['Subj02','Subj03','subj04','Subj05','Subj09','Subj11',
                'Subj12','Subj13','Subj14','Subj15','Subj16','Subj17',
                'Subj18','Subj19','Subj20','Subj22','Subj23','Subj24',
                'Subj26','Subj27','Subj28','Subj29','Subj30','Subj31',
                'Subj32','Subj34','Subj36','Subj37','Subj38','Subj39',
                'Subj40','Subj42']

for subj in subject_list:
    set_path(subj)
    make_bem(freesurfer_path)

In [ ]:
########## Make Noise Covariance ##########
empty_file_list = []
for file in listdir(empty_room_path+'\\raw'):
    if file.endswith('.fif'):
        empty_file_list.append(file)

for f in empty_file_list:
    empty_file = mne.io.read_raw_fif(os.path.join(empty_room_path+'\\raw', f))
    
    ########## Filter Data ########## 
    empty_file = filter_power_noise(empty_file)
    empty_file = filter_band(empty_file)
    ########## Smooth Data ########## 
    empty_file = smooth_savgol_raw(empty_file)
    #empty_file = smooth_butter_raw(empty_file)
    ########## Downsample Data ########## 
    empty_file = downsample_raw(empty_file)
    ########## Pick MEG ##########
    empty_file.pick_types(meg=True, eeg=False, stim=False, misc=False, eog=False, ecg=False, include=[], exclude=[])
    
    make_cov_empty(empty_file,f)

In [ ]:
########## Import Data and Information ########## 
set_path(subj)
raws = import_data()
subject_info = import_info()
modify_raw_obj(raws,subject_info)

for r in range(n_runs):
    ########## Filter Data ########## 
    print("************************* Power-line noise filter - run: "+str(r+1)+" *************************")
    raws[r] = filter_power_noise(raws[r])

    print("************************* Band-pass filter - run: "+str(r+1)+" *************************")
    raws[r] = filter_band(raws[r])
    
    ########## Run ICA ########## 
    if "ica" in tail_fname:
        print("************************* Run ICA - run: "+str(r+1)+" *************************")
        raws[r] = fit_ica(raws[r],r)
    
    ########## Annotate Muscle ########## 
    print("************************* Annotate muscle activity - run: "+str(r+1)+" *************************")
    raws[r] = gen_annot_muscle(raws[r])

    ########## Smooth Data ########## 
    print("************************* Smooth data - run: "+str(r+1)+" *************************")
    raws[r] = smooth_savgol_raw(raws[r])
    #raws[r] = smooth_butter_raw(raws[r])

    ########## Downsample Data ########## 
    print("************************* Downsample data - run: "+str(r+1)+" *************************")
    raws[r] = downsample_raw(raws[r])
    
    ########## Pick MEG ##########
    raws[r].pick_types(meg=True, eeg=False, stim=True, misc=True, eog=False, ecg=False, include=[], exclude=[])

########## Concatenate Data ##########
print("************************* Concatenating raws *************************")
raw = concatenate_raw_obj(raws,on_mismatch='warn')

########## Create Events, Epochs, and Evoked Data ##########
events = make_event(raw)
epochs = epoch_data(events,raw)

epochs['all'].save(meg_path+'\\'+epoched_fname)

In [ ]:
########## Make Coregistration (GUI) ########## 
set_path(subj)
make_coregistration_gui(meg_path+'\\'+epoched_fname)

In [ ]:
########## Make Coregistration (Automatic) ##########
set_path(subj)
make_coregistration_automatic(meg_path+'\\'+epoched_fname)

In [ ]:
########## Make Source Localization ##########
noise_cov_type = "baseline"   #"baseline", "empty"

subject_list = ['Subj02','Subj03','subj04','Subj05','Subj09','Subj11',
                'Subj12','Subj13','Subj14','Subj15','Subj16','Subj17',
                'Subj18','Subj19','Subj20','Subj22','Subj23','Subj24',
                'Subj26','Subj27','Subj28','Subj29','Subj30','Subj31',
                'Subj32','Subj34','Subj36','Subj37','Subj38','Subj39',
                'Subj40','Subj42']

for subj in subject_list:
    set_path(subj)
    subject_info = import_info()
    
    path = os.path.join(meg_path,epoched_fname)
    epochs = mne.read_epochs(path,preload=False)

    if not os.path.exists(volume_path): os.makedirs(volume_path)
    if noise_cov_type == "empty": noise_cov = read_cov_empty(subject_info)
    
    for e in event_sample_list:
        if noise_cov_type == "baseline": noise_cov = make_cov_baseline(figure_path,e,epochs)
        
        stc, src = make_source_localization(epochs[e],epochs[e].average(),noise_cov)
        stc.save(volume_path+'\\'+f"{e}{tail_fname}")
        mne.write_source_spaces(volume_path+'\\'+f"{e}{tail_fname}-src.fif",src)
        #save_surface_src_volume(stc, subject='freesurfer', subjects_dir=mri_folder, time_downsample_factor= 20, save_file_name=volume_path+'\\'+f"stc_{e}.nii.gz")

In [ ]:
########## Visualize Source Localization ########## 
stc = mne.read_source_estimate(volume_path+'\\'+'N1S1TFA1'+tail_fname, subject='freesurfer')
visualize_source_localization(stc,'rh')

In [ ]:
########## Visualize Source Localization (Morph) ########## 
stc = mne.read_source_estimate(meg_path+'\\'+'N4S4TFA2', subject='freesurfer')
visualize_source_localization_morph(stc,'rh')